In [18]:
import sys

sys.path.append('..')

import os
import argparse
import pandas as pd
from natsort import natsorted
from qiskit import QuantumCircuit

BENCHMARK_DPATH = '../benchmarks/uccsd_qasm'
OUTPUT_DPATH = './output_uccsd/'

compiler = 'tket'


OUTPUT_DPATH = os.path.join(OUTPUT_DPATH, compiler)

if not os.path.exists(OUTPUT_DPATH):
    raise ValueError('{} deos not exist'.format(OUTPUT_DPATH))

result_fname = 'result_uccsd_{}.csv'.format(compiler)


# result = pd.DataFrame(columns=['program',
#                                'num_gates(all2all)', 'num_2q_gates(all2all)',
#                                'depth(all2all)', 'depth_2q(all2all)',
#                                'num_gates(manhattan)', 'num_2q_gates(manhattan)',
#                                'depth(manhattan)', 'depth_2q(manhattan)',
#                                'num_gates(sycamore)', 'num_2q_gates(sycamore)',
#                                'depth(sycamore)', 'depth_2q(sycamore)'])

results = {
    'all2all': pd.DataFrame(columns=['program', 'num_gates', 'num_2q_gates', 'depth', 'depth_2q']),
    'manhattan': pd.DataFrame(columns=['program', 'num_gates', 'num_2q_gates', 'depth', 'depth_2q']),
    'sycamore': pd.DataFrame(columns=['program', 'num_gates', 'num_2q_gates', 'depth', 'depth_2q'])
}


for device in ['all2all', 'manhattan', 'sycamore']:
    output_dpath = os.path.join(OUTPUT_DPATH, device)
    print('Processing', output_dpath)
    for fname in natsorted(os.listdir(output_dpath)):
        program_name = fname.split('.')[0]
        output_circ_file = os.path.join(output_dpath, fname)
        circ = QuantumCircuit.from_qasm_file(output_circ_file)

        results[device] = pd.concat([results[device], pd.DataFrame({
            'program': program_name,
            'num_gates': circ.size(),
            'num_2q_gates': circ.num_nonlocal_gates(),
            'depth': circ.depth(),
            'depth_2q': circ.depth(lambda instr: instr.operation.num_qubits > 1)
        }, index=[0])], ignore_index=True)


Processing ./output_uccsd/tket/all2all
Processing ./output_uccsd/tket/manhattan
Processing ./output_uccsd/tket/sycamore


,program,num_gates,num_2q_gates,depth,depth_2q
0,CH2_cmplt_BK_sto3g,34158,26468,19456,16490
1,CH2_cmplt_JW_sto3g,31952,24067,18345,15117
2,CH2_cmplt_P_sto3g,39052,31363,21645,18710
3,CH2_frz_BK_sto3g,18666,14187,11124,9254
4,CH2_frz_JW_sto3g,14212,10840,8439,6992
5,CH2_frz_P_sto3g,19427,15190,10984,9300
6,H2O_cmplt_BK_sto3g,24274,18428,13968,11594
7,H2O_cmplt_JW_sto3g,22605,17607,13428,11344
8,H2O_cmplt_P_sto3g,29215,23407,16901,14472
9,H2O_frz_BK_sto3g,14853,10944,9045,7378


In [45]:

# initially, this data frame contains information of original circuits
result = pd.DataFrame(columns=['program', 'num_qubits', 'num_gates', 'num_2q_gates', 'depth', 'depth_2q'])
for fname in natsorted(os.listdir(BENCHMARK_DPATH)):
    program_name = fname.split('.')[0]
    qasm_file = os.path.join(BENCHMARK_DPATH, fname)
    circ = QuantumCircuit.from_qasm_file(qasm_file)

    result = pd.concat([result, pd.DataFrame({
        'program': program_name,
        'num_qubits': circ.num_qubits,
        'num_gates': circ.size(),
        'num_2q_gates': circ.num_nonlocal_gates(),
        'depth': circ.depth(),
        'depth_2q': circ.depth(lambda instr: instr.operation.num_qubits > 1)
    }, index=[0])], ignore_index=True)

In [46]:
result

,program,num_qubits,num_gates,num_2q_gates,depth,depth_2q
0,CH2_cmplt_BK_sto3g,14,37780,19574,23568,19399
1,CH2_cmplt_JW_sto3g,14,34280,21072,23700,19749
2,CH2_cmplt_P_sto3g,14,40198,20514,24893,20455
3,CH2_frz_BK_sto3g,12,19880,10228,12559,10174
4,CH2_frz_JW_sto3g,12,17658,10344,11914,9706
5,CH2_frz_P_sto3g,12,19532,9938,12346,9889
6,H2O_cmplt_BK_sto3g,14,25238,13108,15797,12976
7,H2O_cmplt_JW_sto3g,14,23210,14360,16264,13576
8,H2O_cmplt_P_sto3g,14,27467,14020,16929,13955
9,H2O_frz_BK_sto3g,12,15624,8004,9691,7934


In [49]:

result = pd.merge(result, results['all2all'], on='program', suffixes=('', '(all2all)'))
result = pd.merge(result, results['manhattan'], on='program', suffixes=('', '(manhattan)'))
result = pd.merge(result, results['sycamore'], on='program', suffixes=('', '(sycamore)'))

In [50]:
result

,program,num_qubits,num_gates,num_2q_gates,depth,depth_2q,num_gates(all2all),num_2q_gates(all2all),depth(all2all),depth_2q(all2all),...,depth(all2all),depth_2q(all2all),num_gates(manhattan),num_2q_gates(manhattan),depth(manhattan),depth_2q(manhattan),num_gates(sycamore),num_2q_gates(sycamore),depth(sycamore),depth_2q(sycamore)
0,CH2_cmplt_BK_sto3g,14,37780,19574,23568,19399,9776,6435,7687,5767,...,7687,5767,34158,26468,19456,16490,22622,17218,14015,11593
1,CH2_cmplt_JW_sto3g,14,34280,21072,23700,19749,9562,6039,6985,5048,...,6985,5048,31952,24067,18345,15117,21541,16082,13802,11197
2,CH2_cmplt_P_sto3g,14,40198,20514,24893,20455,10829,7345,8482,6538,...,8482,6538,39052,31363,21645,18710,56221,20937,27409,14423
3,CH2_frz_BK_sto3g,12,19880,10228,12559,10174,5685,3640,4438,3272,...,4438,3272,18666,14187,11124,9254,25706,9566,12988,6826
4,CH2_frz_JW_sto3g,12,17658,10344,11914,9706,4724,2916,3595,2542,...,3595,2542,14212,10840,8439,6992,20099,7481,9897,5216
5,CH2_frz_P_sto3g,12,19532,9938,12346,9889,5792,3791,4487,3380,...,4487,3380,19427,15190,10984,9300,26982,10099,13264,6989
6,H2O_cmplt_BK_sto3g,14,25238,13108,15797,12976,7304,4680,5626,4135,...,5626,4135,24274,18428,13968,11594,34132,12614,16751,8804
7,H2O_cmplt_JW_sto3g,14,23210,14360,16264,13576,6618,4299,5039,3746,...,5039,3746,22605,17607,13428,11344,32008,11782,15264,7994
8,H2O_cmplt_P_sto3g,14,27467,14020,16929,13955,8054,5458,6412,4939,...,6412,4939,29215,23407,16901,14472,42437,15692,20628,10869
9,H2O_frz_BK_sto3g,12,15624,8004,9691,7934,4716,2947,3594,2615,...,3594,2615,14853,10944,9045,7378,20745,7668,10255,5383


In [ ]:
r1 = pd.merge(result, results['all2all'], on='program', suffixes=('', '(all2all)'))

,program,num_qubits,num_gates,num_2q_gates,depth,depth_2q,num_gates(all2all),num_2q_gates(all2all),depth(all2all),depth_2q(all2all)
0,CH2_cmplt_BK_sto3g,14,37780,19574,23568,19399,9776,6435,7687,5767
1,CH2_cmplt_JW_sto3g,14,34280,21072,23700,19749,9562,6039,6985,5048
2,CH2_cmplt_P_sto3g,14,40198,20514,24893,20455,10829,7345,8482,6538
3,CH2_frz_BK_sto3g,12,19880,10228,12559,10174,5685,3640,4438,3272
4,CH2_frz_JW_sto3g,12,17658,10344,11914,9706,4724,2916,3595,2542
5,CH2_frz_P_sto3g,12,19532,9938,12346,9889,5792,3791,4487,3380
6,H2O_cmplt_BK_sto3g,14,25238,13108,15797,12976,7304,4680,5626,4135
7,H2O_cmplt_JW_sto3g,14,23210,14360,16264,13576,6618,4299,5039,3746
8,H2O_cmplt_P_sto3g,14,27467,14020,16929,13955,8054,5458,6412,4939
9,H2O_frz_BK_sto3g,12,15624,8004,9691,7934,4716,2947,3594,2615


In [10]:
r2 = pd.merge(r1, results['manhattan'], on='program', suffixes=('', '(manhattan)'))

In [12]:
r2

,program,num_qubits,num_gates,num_2q_gates,depth,depth_2q,num_gates(all2all),num_2q_gates(all2all),depth(all2all),depth_2q(all2all),num_gates(manhattan),num_2q_gates(manhattan),depth(manhattan),depth_2q(manhattan)
0,CH2_cmplt_BK_sto3g,14,37780,19574,23568,19399,9776,6435,7687,5767,34158,26468,19456,16490
1,CH2_cmplt_JW_sto3g,14,34280,21072,23700,19749,9562,6039,6985,5048,31952,24067,18345,15117
2,CH2_cmplt_P_sto3g,14,40198,20514,24893,20455,10829,7345,8482,6538,39052,31363,21645,18710
3,CH2_frz_BK_sto3g,12,19880,10228,12559,10174,5685,3640,4438,3272,18666,14187,11124,9254
4,CH2_frz_JW_sto3g,12,17658,10344,11914,9706,4724,2916,3595,2542,14212,10840,8439,6992
5,CH2_frz_P_sto3g,12,19532,9938,12346,9889,5792,3791,4487,3380,19427,15190,10984,9300
6,H2O_cmplt_BK_sto3g,14,25238,13108,15797,12976,7304,4680,5626,4135,24274,18428,13968,11594
7,H2O_cmplt_JW_sto3g,14,23210,14360,16264,13576,6618,4299,5039,3746,22605,17607,13428,11344
8,H2O_cmplt_P_sto3g,14,27467,14020,16929,13955,8054,5458,6412,4939,29215,23407,16901,14472
9,H2O_frz_BK_sto3g,12,15624,8004,9691,7934,4716,2947,3594,2615,14853,10944,9045,7378


In [13]:
pd.merge(r2, results['sycamore'], on='program', suffixes=('', '(sycamore)'))

,program,num_qubits,num_gates,num_2q_gates,depth,depth_2q,num_gates(all2all),num_2q_gates(all2all),depth(all2all),depth_2q(all2all),num_gates(manhattan),num_2q_gates(manhattan),depth(manhattan),depth_2q(manhattan),num_gates(sycamore),num_2q_gates(sycamore),depth(sycamore),depth_2q(sycamore)
0,CH2_cmplt_BK_sto3g,14,37780,19574,23568,19399,9776,6435,7687,5767,34158,26468,19456,16490,22622,17218,14015,11593
1,CH2_cmplt_JW_sto3g,14,34280,21072,23700,19749,9562,6039,6985,5048,31952,24067,18345,15117,21541,16082,13802,11197
2,CH2_cmplt_P_sto3g,14,40198,20514,24893,20455,10829,7345,8482,6538,39052,31363,21645,18710,56221,20937,27409,14423
3,CH2_frz_BK_sto3g,12,19880,10228,12559,10174,5685,3640,4438,3272,18666,14187,11124,9254,25706,9566,12988,6826
4,CH2_frz_JW_sto3g,12,17658,10344,11914,9706,4724,2916,3595,2542,14212,10840,8439,6992,20099,7481,9897,5216
5,CH2_frz_P_sto3g,12,19532,9938,12346,9889,5792,3791,4487,3380,19427,15190,10984,9300,26982,10099,13264,6989
6,H2O_cmplt_BK_sto3g,14,25238,13108,15797,12976,7304,4680,5626,4135,24274,18428,13968,11594,34132,12614,16751,8804
7,H2O_cmplt_JW_sto3g,14,23210,14360,16264,13576,6618,4299,5039,3746,22605,17607,13428,11344,32008,11782,15264,7994
8,H2O_cmplt_P_sto3g,14,27467,14020,16929,13955,8054,5458,6412,4939,29215,23407,16901,14472,42437,15692,20628,10869
9,H2O_frz_BK_sto3g,12,15624,8004,9691,7934,4716,2947,3594,2615,14853,10944,9045,7378,20745,7668,10255,5383


In [30]:

result = pd.merge(result, results['all2all'], on='program', suffixes=('', '(all2all)'))
reuslt = pd.merge(result, results['manhattan'], on='program', suffixes=('', '(manhattan)'))
result = pd.merge(result, results['sycamore'], on='program', suffixes=('', '(sycamore)'))

In [31]:
results['manhattan']

,program,num_gates,num_2q_gates,depth,depth_2q
0,CH2_cmplt_BK_sto3g,34158,26468,19456,16490
1,CH2_cmplt_JW_sto3g,31952,24067,18345,15117
2,CH2_cmplt_P_sto3g,39052,31363,21645,18710
3,CH2_frz_BK_sto3g,18666,14187,11124,9254
4,CH2_frz_JW_sto3g,14212,10840,8439,6992
5,CH2_frz_P_sto3g,19427,15190,10984,9300
6,H2O_cmplt_BK_sto3g,24274,18428,13968,11594
7,H2O_cmplt_JW_sto3g,22605,17607,13428,11344
8,H2O_cmplt_P_sto3g,29215,23407,16901,14472
9,H2O_frz_BK_sto3g,14853,10944,9045,7378


In [32]:
# reuslt = pd.merge(result, results['sycamore'], on='program', suffixes=('', '(sycamore)'))

result

,program,num_qubits,num_gates,num_2q_gates,depth,depth_2q,num_gates(all2all),num_2q_gates(all2all),depth(all2all),depth_2q(all2all),num_gates(all2all),num_2q_gates(all2all),depth(all2all),depth_2q(all2all),num_gates(sycamore),num_2q_gates(sycamore),depth(sycamore),depth_2q(sycamore)
0,CH2_cmplt_BK_sto3g,14,37780,19574,23568,19399,9776,6435,7687,5767,9776,6435,7687,5767,22622,17218,14015,11593
1,CH2_cmplt_JW_sto3g,14,34280,21072,23700,19749,9562,6039,6985,5048,9562,6039,6985,5048,21541,16082,13802,11197
2,CH2_cmplt_P_sto3g,14,40198,20514,24893,20455,10829,7345,8482,6538,10829,7345,8482,6538,56221,20937,27409,14423
3,CH2_frz_BK_sto3g,12,19880,10228,12559,10174,5685,3640,4438,3272,5685,3640,4438,3272,25706,9566,12988,6826
4,CH2_frz_JW_sto3g,12,17658,10344,11914,9706,4724,2916,3595,2542,4724,2916,3595,2542,20099,7481,9897,5216
5,CH2_frz_P_sto3g,12,19532,9938,12346,9889,5792,3791,4487,3380,5792,3791,4487,3380,26982,10099,13264,6989
6,H2O_cmplt_BK_sto3g,14,25238,13108,15797,12976,7304,4680,5626,4135,7304,4680,5626,4135,34132,12614,16751,8804
7,H2O_cmplt_JW_sto3g,14,23210,14360,16264,13576,6618,4299,5039,3746,6618,4299,5039,3746,32008,11782,15264,7994
8,H2O_cmplt_P_sto3g,14,27467,14020,16929,13955,8054,5458,6412,4939,8054,5458,6412,4939,42437,15692,20628,10869
9,H2O_frz_BK_sto3g,12,15624,8004,9691,7934,4716,2947,3594,2615,4716,2947,3594,2615,20745,7668,10255,5383
